In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from crewai import Agent,LLM,Crew,Task
from crewai_tools import TavilySearchTool
from crewai.tools import tool

In [4]:
llm = LLM(model="groq/llama-3.3-70b-versatile", temperature=0.2)

In [7]:
search_tool = TavilySearchTool(max_results=2,topic="finance")

In [ ]:
market_analysis_agent = Agent(
    role="Senior Stock Market Analyst",
    goal="Perform deep financial and market analysis for the stock symbol {symbol}.",
    backstory="""
    You are a senior equity research analyst with 15+ years of experience.
    You specialize in analyzing company financials, profitability, valuation metrics,
    revenue growth, earnings reports, and overall market trends.
    You provide structured, professional financial insights.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [ ]:
future_trend_agent = Agent(
    role="Investment Forecast Strategist",
    goal="Predict short-term future outlook and hype potential of stock {symbol}.",
    backstory="""
    You are a financial forecasting expert skilled in analyzing
    market sentiment, stock momentum, macroeconomic signals,
    institutional activity, and growth catalysts.
    You provide realistic, probability-based forecasts.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [ ]:
risk_analysis_agent = Agent(
    role="Financial Risk Assessment Specialist",
    goal="Identify risks, weaknesses, and potential threats for stock {symbol}.",
    backstory="""
    You are a risk auditor who identifies volatility risks,
    regulatory threats, competition challenges, lawsuits,
    declining revenues, and economic pressure.
    You focus on downside protection analysis.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [ ]:
report_aggregator_agent = Agent(
    role="Chief Investment Report Strategist",
    goal="Aggregate all stock analysis and generate final professional report for {symbol}.",
    backstory="""
    You are a senior financial strategist responsible for preparing
    executive-level stock evaluation reports.
    You combine multiple expert analyses into one clear,
    structured and actionable investment report.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [ ]:
market_analysis_task = Task(
    description="""
    Perform detailed financial and market analysis for stock symbol {symbol}.

    Include:
    - Current market trend
    - Revenue and profit performance
    - Financial metrics (P/E, EPS, Market Cap if available)
    - Recent quarterly performance
    - Overall financial health
    """,
    expected_output="Structured financial analysis report.",
    agent=market_analysis_agent
)


In [ ]:
future_trend_task = Task(
    description="""
    Analyze the future outlook of stock {symbol}.

    Include:
    - Short-term prediction (next 2 months)
    - Growth potential
    - Market hype level (High/Moderate/Low)
    - Investment attractiveness
    """,
    expected_output="Future investment outlook report.",
    agent=future_trend_agent
)


In [ ]:
risk_analysis_task = Task(
    description="""
    Perform risk and downside analysis for stock {symbol}.

    Include:
    - Recent losses
    - Industry competition threats
    - Regulatory risks
    - Market volatility
    - Any current issues
    """,
    expected_output="Detailed risk analysis report.",
    agent=risk_analysis_agent
)


In [ ]:
final_report_task = Task(
    description="""
    Using all previous analyses, generate a comprehensive investment report for {symbol}.

    The report must include:

    1. Executive Summary
    2. Market Performance Overview
    3. Future Outlook (Next 2 Months)
    4. Risk Assessment
    5. Final Recommendation (Buy / Hold / Sell with justification)

    Make it professional and investor-ready.
    """,
    expected_output="Complete stock evaluation report.",
    agent=report_aggregator_agent,
    context=[
        market_analysis_task,
        future_trend_task,
        risk_analysis_task
    ]
)


In [ ]:
crew = Crew(
    agents=[
        market_analysis_agent,
        future_trend_agent,
        risk_analysis_agent,
        report_aggregator_agent
    ],
    tasks=[
        market_analysis_task,
        future_trend_task,
        risk_analysis_task,
        final_report_task
    ],
    process=Process.sequential,
    verbose=True
)


In [ ]:
result = crew.kickoff(inputs={"symbol": "AAPL"})   # inputs={"symbol": "TSLA"}
print(result)
